In [17]:
#imports
import musicbrainzngs
import pandas as pd

In [18]:
# Set the user agent for your application
musicbrainzngs.set_useragent("Mozilla/5.0 (X11; Linux x86_64; rv:143.0) Gecko/20100101 Firefox/143.0", "0.5", "bittah.pupil@proton.me")

In [19]:
album_title = "Dune"  # Replace with your album title

try:
    results = musicbrainzngs.search_releases(release = "NightFlight", artist = "Gabor Szabo", strict = True)
    if not results['release-list']:
        print(f"No releases found for '{album_title}'.")
        exit()
    
    # Assuming the first result is the desired album, or you can add logic to choose
    album_id = results['release-list'][0]['id']
    print(f"Found album '{results['release-list'][0]['title']}' with ID: {album_id}")
except musicbrainzngs.WebServiceError as e:
    print(f"Error searching for releases: {e}")
    exit()

Found album 'Nightflight' with ID: 0dc9afcb-c3de-4558-8c4e-e56604152ff6


In [25]:
for release in results['release-list']:
    print(f"Title: {release['title']}, Date: {release['date']}, Artist Name: {release['artist-credit'][0]['name']}, Mbid: {release['id']}")

Title: Nightflight, Date: 1976, Artist Name: Gábor Szabó, Mbid: 0dc9afcb-c3de-4558-8c4e-e56604152ff6
Title: Nightflight, Date: 2015-01-16, Artist Name: Gábor Szabó, Mbid: f9d4d610-0592-4195-b5c5-61d3dba819dc


In [21]:
results

{'release-list': [{'id': '0dc9afcb-c3de-4558-8c4e-e56604152ff6',
   'ext:score': '100',
   'title': 'Nightflight',
   'status': 'Official',
   'text-representation': {'language': 'eng', 'script': 'Latn'},
   'artist-credit': [{'name': 'Gábor Szabó',
     'artist': {'id': 'ee69197a-ca57-42ba-bd23-93b756391a5d',
      'name': 'Gábor Szabó',
      'sort-name': 'Szabó, Gábor'}}],
   'release-group': {'id': '94f41074-662f-3314-9ba2-40c512a558f9',
    'type': 'Album',
    'title': 'Nightflight',
    'primary-type': 'Album'},
   'date': '1976',
   'country': 'US',
   'release-event-list': [{'date': '1976',
     'area': {'id': '489ce91b-6658-3307-9877-795b68554c98',
      'name': 'United States',
      'sort-name': 'United States',
      'iso-3166-1-code-list': ['US']}}],
   'label-info-list': [{'catalog-number': 'SRM 1-1091',
     'label': {'id': '995428e7-81b6-41dd-bd38-5a7a0ece8ad6',
      'name': 'Mercury Records'}}],
   'medium-list': [{'format': 'Vinyl',
     'disc-list': [],
     'disc-